## Check the gas cube in FIRE simulation

In [1]:
# import packages
from astropy.utils import data
from radio_beam import Beam
from astropy import units as u
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
from scipy.stats.stats import pearsonr
from matplotlib import rcParams
%matplotlib inline
rcParams['mathtext.default']='regular'
import seaborn as sns
from scipy import stats
from mpl_toolkits.axes_grid1 import make_axes_locatable
from math import log10, floor

# import functions
def count_time(stop, start):
    '''
    Convert the time difference into human readable form. 
    '''
    dure=stop-start
    m,s=divmod(dure,60)
    h,m=divmod(m,60)
    print("%d:%02d:%02d" %(h, m, s))

    return

def make_mom0(dataCubes, pixsize=100, value=1e10):
    '''
    ------
    Parameters
    dataCubes: 3d numpy array
        3D numpy array to make moment maps
    pixsize: float
        Size of each pixel in pc
    value: float
        pixel unit in solar mass
    '''
    mom0 = np.nansum(dataCubes, axis=2) * value / pixsize**2
    
    return mom0
    
def make_mom1(dataCubes, vel_1d):
    '''
    ------
    Parameters 
    dataCubes: 3d numpy array
        3D numpy array to make moment maps
    vel_1d: 1d numpy array
        1D array of velocity values corresponding to the 3rd
        axis of the dataCubes. 
    '''          
    vel_3d = np.full(np.shape(dataCubes),fill_value=np.nan)
    vel_3d[:] = vel_1d
    mom1 = np.nansum(vel_3d*dataCubes,axis=2) / np.nansum(dataCubes,axis=2)
              
    return mom1

def make_mom2(dataCubes, vel_1d):
    '''
    ------
    Parameters
    dataCubes: 3d numpy array
        3D numpy array to make moment maps
    vel_1d: 1d numpy array
        1D array of velocity values corresponding to the 3rd
        axis of the dataCubes.
    '''
    vel_3d = np.full(np.shape(dataCubes),fill_value=np.nan)
    vel_3d[:] = vel_1d
    mom1 = np.nansum(vel_3d*dataCubes,axis=2) / np.nansum(dataCubes,axis=2)
    
    mom1_3d = np.repeat(mom1[:,:,np.newaxis], len(vel_1d), axis=2)
    vel_diff = vel_3d - mom1_3d
    mom2 = np.sqrt(np.nansum(vel_diff**2*dataCubes,axis=2) / np.nansum(dataCubes,axis=2))

    return mom2

def make_Tpeak(dataCubes, pixsize=100, value=1e10, alphaCO=4.3, ratio=0.7, deltaV=2):
    '''
    ------
    Parameters
    dataCubes: 3d numpy array
        3D numpy array to make moment maps
    pixsize: float
        Size of each pixel in pc
    value: float
        pixel unit in solarmass
    alphaCO: float
        CO-H2 conversion factor
    ratio: float
        CO 2-1/1-0 ratio
    deltaV : float
        Velocity resolution per channel
    '''
    mom8 = np.nanmax(dataCubes, axis=2)*value/pixsize**2
    Tpeak = mom8 / alphaCO / deltaV * ratio
    Tpeak[np.where(np.isnan(Tpeak))] = 0
    
    return Tpeak

def round_sig(x, sig=2):
    '''
    round the x to certain significant figures
    '''
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [2]:
npzDir = '../FIRE_GalSep/'
picDir = '../pictures/'
fitsDir = '../data/e2/'

### Make the animation for the change vdep-Sigmol plot as a function of the time

#### Make the animation

In [9]:
%matplotlib inline
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from random import randint
import matplotlib.gridspec as gridspec
snapshotsPart = np.arange(520, 780, 10)

# load the SFR history information
filename=npzDir+'galsep_G2G3_e_orbit_2.npz'
data = np.load(filename)
print('Entry names:'+str(data.files))
SFRs = data['sfr']
Times = data['time']
Snapshots = data['isnap']

incl = 'v0'

fig = plt.figure(figsize=(9,8))
spec = gridspec.GridSpec(2, 1, height_ratios=[1,3])
# second gridspec
gc1 = gridspec.GridSpecFromSubplotSpec(ncols=1, nrows=1, subplot_spec=spec[0])
ax1 = fig.add_subplot(gc1[0], adjustable='box', aspect=0.02)
# first gridspec
gc0 = gridspec.GridSpecFromSubplotSpec(ncols=1, nrows=1, subplot_spec=spec[1])
ax0 = fig.add_subplot(gc0[0])

# make the animation
def animate(i):
    # plot the Sigmol versus velocity dispersion
    ax0.clear()
    idNo = snapshotsPart[i]
    image=plt.imread(picDir+'Sigmol_vdep_G2G3_e2_'+str(idNo)+'_'+str(incl)+'.png')
    ax0.imshow(image)
    ax0.set_xticks([])
    ax0.set_yticks([])
    # label the time
    Time = Times[np.where(Snapshots==idNo)][0]
    ax0.annotate(str(incl)+': '+str(round_sig(Time, sig=3))+' Gyr', (0.2, 0.9), 
                 xycoords='axes fraction', fontsize=15)
    
    # plot the SFR versus time
    ax1.clear()
    idNo = snapshotsPart[i]
    ax1.plot(Times, SFRs)
    ax1.axvline(Times[np.where(Snapshots==idNo)], color='black')
    ax1.set_xlabel('time (Gyr)', fontsize=15)
    ax1.set_ylabel(r'SFR (M$_{\odot}$ yr$^{-1}$', fontsize=15)

anim = FuncAnimation(fig, animate, frames=26, interval=500, repeat=False)
plt.close()
from IPython.display import HTML
HTML(anim.to_jshtml())

# giffile = picDir+'Sigmol_vdep_G2_3_e2.gif'
# writergif = animation.PillowWriter(fps=1)
# anim.save(giffile, writer=writergif, dpi=100)

### Make the animation including the moment maps

In [1]:
%matplotlib inline
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from random import randint
import matplotlib.gridspec as gridspec
snapshotsPart = np.arange(520, 712, 2)

incl = 'v0'

fig = plt.figure(figsize=(15,8))
spec = gridspec.GridSpec(1, 2, width_ratios=[1.5,1])
# first gridspec
gc0 = gridspec.GridSpecFromSubplotSpec(ncols=1, nrows=1, subplot_spec=spec[0])
ax0 = fig.add_subplot(gc0[0])
# second gridspec
gc1 = gridspec.GridSpecFromSubplotSpec(ncols=1, nrows=2, subplot_spec=spec[1], height_ratios=[1, 4])
ax1 = fig.add_subplot(gc1[0])
ax2 = fig.add_subplot(gc1[1])

# make the animation
def animate(i):
    # plot the Sigmol versus velocity dispersion
    ax0.clear()
    idNo = snapshotsPart[i]
    image=plt.imread(picDir+'Sigmol_vdep_G2G3_e2_'+str(idNo)+'_'+str(incl)+'.png')
    ax0.imshow(image)
    ax0.set_xticks([])
    ax0.set_yticks([])
    # label the time
    Time = Times[np.where(Snapshots==idNo)][0]
    ax0.annotate(str(incl)+': '+str(round_sig(Time, sig=3))+' Gyr', (0.2, 0.9), 
                 xycoords='axes fraction', fontsize=15)
    
    # plot the SFR versus time
    ax1.clear()
    idNo = snapshotsPart[i]
    ax1.plot(Times, SFRs)
    ax1.axvline(Times[np.where(Snapshots==idNo)], color='black')
    ax1.set_xlabel('time (Gyr)', fontsize=15)
    ax1.set_ylabel(r'SFR (M$_{\odot}$ yr$^{-1}$', fontsize=15)
    
    # plot the moment map variation
    ax2.clear()
    idNo = snapshotsPart[i]
    image=plt.imread(picDir+'G2G3_e2_mom0_'+str(idNo)+'_'+str(incl)+'.png')
    ax2.imshow(image)
    ax2.set_xticks([])
    ax2.set_yticks([])
    # label the time
    Time = Times[np.where(Snapshots==idNo)][0]
    ax2.annotate(str(incl)+': '+str(round_sig(Time, sig=3))+' Gyr', (0.13, 0.82), 
                 xycoords='axes fraction', fontsize=15)
                    
anim = FuncAnimation(fig, animate, frames=96, interval=100, repeat=False)
plt.close()
from IPython.display import HTML
HTML(anim.to_jshtml())

# # save as gif
# giffile = picDir+'Sigmol_vdep_G2_3_e2_'+str(incl)+'.gif'
# writergif = animation.PillowWriter(fps=3)
# anim.save(giffile, writer=writergif, dpi=100)

NameError: name 'np' is not defined

In [8]:
# save as mp4
mp4file = picDir+'Sigmol_vdep_G2_3_e2_'+str(incl)+'.mp4'
writermp4 = animation.FFMpegWriter(fps=3)
anim.save(mp4file, writer=writermp4)
fig.savefig(picDir+'Sigmol_vdep_example.pdf')